In [ ]:
# default_exp tasks.vos

In [ ]:
#hide
from nbdev.showdoc import *

# [Beta] Virtual Outlier Synthesis
> Means to train a fastai model using [Virtual Outlier Synthesis](https://arxiv.org/abs/2202.01197).

VOS requires the metrics refactor from this draft PR to work. You need to pip install from git.

```
pip install git+https://github.com/warner-benjamin/fastai.git@metrics_rewrite
```

In [ ]:
#exporti

# Original VOS implementation from https://github.com/deeplearning-wisc/vos

# Original code, Copyright 2022 by Xuefeng Du, Zhaoning Wang, Mu Cai & Yixuan Li
# Licensed under the Apache License, Version 2.0

In [ ]:
#export
from __future__ import annotations
import numpy as np
from fastai.learner import Learner, replacing_yield
from fastai.torch_core import Module, defaults, default_device
from fastai.callback.core import Callback
from fastai.callback.fp16 import MixedPrecision
from fastai.data.transforms import get_c
from fastcore.basics import store_attr, ifnone
from fastcore.foundation import L, mk_class, patch
from fastcore.meta import delegates, contextmanager
from torch.distributions.multivariate_normal import MultivariateNormal
from fastai.vision.learner import create_body, num_features_model, apply_init, model_meta, _default_meta
from fastai.losses import CrossEntropyLossFlat, LabelSmoothingCrossEntropy, LabelSmoothingCrossEntropyFlat
from fastai.layers import *
from fastxtend.imports import *

## Temporary Learner Stuff -

In [ ]:
#exporti
from fastai.basics import Path
from fastai.optimizer import Adam
from fastai.learner import trainable_params, CancelFitException
from fastcore.xtras import ContextManagers

# This section adds LearnerState to `Learner` via patches, which currently is a draft pr in the fastai repo.

In [ ]:
#exporti
mk_class('LearnerState', **{o:o.lower() for o in ['Fit', 'Eval', 'LRFind', 'Idle']},
         doc="All built in states for `Learner")

In [ ]:
#exporti
@patch
def __init__(self:Learner, dls, model, loss_func=None, opt_func=Adam, lr=defaults.lr, splitter=trainable_params, cbs=None,
                metrics=None, path=None, model_dir='models', wd=None, wd_bn_bias=False, train_bn=True,
                moms=(0.95,0.85,0.95)):
    path = Path(path) if path is not None else getattr(dls, 'path', Path('.'))
    if loss_func is None:
        loss_func = getattr(dls.train_ds, 'loss_func', None)
        assert loss_func is not None, "Could not infer loss function from the data, please pass a loss function."
    self.dls,self.model = dls,model
    store_attr(but='dls,model,cbs')
    self.training,self.create_mbar,self.logger,self.opt,self.cbs = False,True,print,None,L()
    self.add_cbs(L(defaults.callbacks)+L(cbs))
    self("after_create")
    self.state=LearnerState.Idle

In [ ]:
#exporti
@patch
@contextmanager
def set_state(self:Learner, state): return replacing_yield(self, 'state', state)

In [ ]:
#exporti
@patch
def fit(self:Learner, n_epoch, lr=None, wd=None, cbs=None, reset_opt=False):
    with self.added_cbs(cbs), self.set_state(LearnerState.Fit):
        if reset_opt or not self.opt: self.create_opt()
        if wd is None: wd = self.wd
        if wd is not None: self.opt.set_hypers(wd=wd)
        self.opt.set_hypers(lr=self.lr if lr is None else lr)
        self.n_epoch = n_epoch
        self._with_events(self._do_fit, 'fit', CancelFitException, self._end_cleanup)

In [ ]:
#exporti
@patch
def validation_context(self:Learner, cbs=None, inner=False):
    cms = [self.no_logging(),self.no_mbar(),self.set_state(LearnerState.Eval)]
    if cbs: cms.append(self.added_cbs(cbs))
    if not inner: cms.append(self)
    return ContextManagers(cms)

## VOS Model Heads -

In [ ]:
#export
class VOSHead(Module):
    def __init__(self, 
        nf:int,
        n_out:int,
        lin_ftrs:list[int]|None=None,
        p:float|None=None,
        concat_pool:bool=True,
        first_bn:bool=True,
        bn_final:bool=False,
        lin_first:bool=False,
        y_range:float|None=None,
        act_cls:Callable[...,nn.Module]=defaults.activation,
        fastai_head:bool=False,
        head_pool:Callable[[int,int],nn.Module]=AdaptiveAvgPool
    ):
        if fastai_head:
            if concat_pool: nf *= 2
            lin_ftrs = [nf, 512, n_out] if lin_ftrs is None else [nf] + lin_ftrs + [n_out]
            bns = [first_bn] + [True]*len(lin_ftrs[1:])
            ps = L(p if p is not None else 0.5)
            if len(ps) == 1: ps = [ps[0]/2] * (len(lin_ftrs)-2) + ps
            actns = [act_cls(inplace=True)] * (len(lin_ftrs)-2) + [None]
            pool = AdaptiveConcatPool2d() if concat_pool else nn.AdaptiveAvgPool2d(1)
            pool_layers = [pool, Flatten()]
            if lin_first: pool_layers.append(nn.Dropout(ps.pop(0)))
            self.pool = nn.Sequential(*pool_layers)
            layers = []
            for ni,no,bn,p,actn in zip(lin_ftrs[:-1], lin_ftrs[1:], bns, ps, actns):
                layers += LinBnDrop(ni, no, bn=bn, p=p, act=actn, lin_first=lin_first)
            if lin_first: layers.append(nn.Linear(lin_ftrs[-2], n_out))
            if bn_final: layers.append(nn.BatchNorm1d(lin_ftrs[-1], momentum=0.01))
            if y_range is not None: layers.append(SigmoidRange(*y_range))
            self.head = nn.Sequential(*layers)
        else:
            p = p if p is not None else 0.0
            self.pool = nn.Sequential(*[head_pool(sz=1, ndim=2), Flatten()])
            self.head = nn.Sequential(*[nn.Dropout(p), nn.Linear(nf, n_out)])
        self.weight_energy = torch.nn.Linear(n_out, 1)
        self.log_reg = torch.nn.Linear(1, 2)
        self.nf = nf

    def forward(self, x):
        x = self.pool(x)
        return self.head(x), x

In [ ]:
#export
@delegates(VOSHead.__init__)
def create_vos_model(
    arch:nn.Module, 
    n_out:int, 
    pretrained:bool=True, 
    cut:int|None=None, 
    n_in:int=3, 
    init:Callable=nn.init.kaiming_normal_, 
    custom_head:nn.Module|None=None,
    concat_pool:bool=True, 
    **kwargs
):
    "Create custom convnet architecture"
    meta = model_meta.get(arch, _default_meta)
    body = create_body(arch, n_in, pretrained, ifnone(cut, meta['cut']))
    if custom_head is None:
        nf = num_features_model(nn.Sequential(*body.children()))
        head = VOSHead(nf, n_out, concat_pool=concat_pool, **kwargs)
    else: head = custom_head
    model = nn.Sequential(body, head)
    if init is not None: apply_init(model[1], init)
    torch.nn.init.uniform_(model[1].weight_energy.weight)
    return model

## VOS Callback -

In [ ]:
#export
mk_class('VOSDistSample', **{o:o.lower() for o in ['List', 'Fork', 'Batch']},
         doc="Sample features via simple for loop, torch.jit.fork for loop, as a batch. Batch will affect variance of samples.")

_all_=['VOSDistSample']

In [ ]:
#exporti
def list_dist(mean_embed_id, temp_precision, sample_from, select, nf, n_out):
    ood_samples = []
    temp_precision = torch.linalg.cholesky(temp_precision)
    for idx in range(n_out):
        new_dis = MultivariateNormal(mean_embed_id, scale_tril=temp_precision)
        negative_samples = new_dis.rsample((sample_from,))
        prob_density = new_dis.log_prob(negative_samples)
        # keep the data in the low density area.
        _, index_prob = torch.topk(-prob_density, select)
        ood_samples.append(negative_samples[index_prob])
    return torch.cat(ood_samples, 0)

In [ ]:
#exporti
def calc_forked_dist(mean_embed_id, temp_precision, sample_from, select):
    new_dis = MultivariateNormal(mean_embed_id, scale_tril=temp_precision)
    negative_samples = new_dis.rsample((sample_from,))
    prob_density = new_dis.log_prob(negative_samples)
    # keep the data in the low density area.
    _, index_prob = torch.topk(-prob_density, select)
    return negative_samples[index_prob]

def forked_dist(mean_embed_id, temp_precision, sample_from, select, nf, n_out):
    temp_precision = torch.linalg.cholesky(temp_precision)
    futures:list[torch.jit.Future[torch.Tensor]] = []
    ood_samples = []
    for idx in range(n_out):
        futures.append(torch.jit.fork(calc_forked_dist, mean_embed_id[idx],
                                      temp_precision, sample_from, select))
    for future in futures:
        ood_samples.append(torch.jit.wait(future))
    return torch.cat(ood_samples, 0)

In [ ]:
#exporti
def batched_dist(mean_embed_id, temp_precision, sample_from, select, nf, n_out):
    new_dis = MultivariateNormal(mean_embed_id, covariance_matrix=temp_precision)
    negative_samples = new_dis.rsample((sample_from,))
    prob_density = new_dis.log_prob(negative_samples)

    # keep the data in the low density area
    _, idx_prob = torch.topk(-prob_density.t(), select)
    negative_samples = negative_samples.permute(1,0,2)
    return negative_samples.gather(dim=1, index=idx_prob.repeat(1,1,nf)).squeeze()

In [ ]:
#export
class VOSCallback(Callback):
    order = MixedPrecision.order-1
    "Base class with shared initializtion for Virtual Outlier Synthesis"
    def __init__(self, 
        n_out:int|None=None, # Number of labels, defaults to `learn.dls.c``
        start_epoch:int|None=None, # Epoch to start Virtual Outlier Synthesis, defaults to `start_pct`
        start_pct:float=0.40, # Iteration percent to start Virtual Outlier Synthesis
        n_sample:int=1000, # Number of virtual outliers to generate
        sample_from:int=10000,
        energy_samples:int=1,
        energy_loss_weight:float=0.1,
        energy_loss_func:nn.Module|None=None,
        dist_sample:DistSample=VOSDistSample.Fork
    ):
        store_attr(but='dist_sample')
        if dist_sample==VOSDistSample.List:    self._sample_dist = list_dist
        elif dist_sample==VOSDistSample.Fork:  self._sample_dist = forked_dist
        elif dist_sample==VOSDistSample.Batch: self._sample_dist = batched_dist

    def before_fit(self):
        assert isinstance(self.learn.model[1], VOSHead), 'VOSCallback requires model have a be `VOSHead'
        if self.learn.state == LearnerState.Fit:
            self.device = device = getattr(self.learn.dls, 'device', default_device())
            self.n_out = ifnone(self.n_out, get_c(self.learn.dls))
            self.nf = self.learn.model[1].nf
            assert self.n_out, "`n_out` is not defined, and could not be inferred from data, set `dls.c` or pass `n_out`"
            self.eye_matrix = torch.eye(self.nf, device=self.device)
            self.features = torch.zeros(self.n_out, self.n_sample, self.nf).to(device=self.device)
            self.number_dict = {}
            for i in range(self.n_out): self.number_dict[i] = 0
            self.sample_count, self.total_samples = 0, self.n_out * self.n_sample
            if isinstance(self.learn.loss_func, (nn.CrossEntropyLoss,CrossEntropyLossFlat,LabelSmoothingCrossEntropy,LabelSmoothingCrossEntropyFlat)):
                self.energy_loss_func = self.learn.loss_func
            elif self.energy_loss_func is None:
                self.energy_loss_func = torch.nn.CrossEntropyLoss()
        self.train_vos = self.learn.state != LearnerState.LRFind
        self.eval_vos  = self.learn.state == LearnerState.Eval
        self.targ_vos  = len(self.learn.dls.valid.one_batch()) - self.learn.dls.valid.n_inp > 1

    def after_pred(self):
        pred, self.batch_features = self.learn.pred
        self.batch_features = self.batch_features.detach().clone()
        self.learn.pred = pred

    def after_loss(self):
        if self.train_vos:
            if self.training: self._vos_train_step()
            else:             self._vos_valid_step()

    def _vos_train_step(self):
        if self.targ_vos:
            self.orig_loss = self.learn.loss.detach().clone()
            targs, self.vos_targs = self.learn.yb
            self.learn.yb = (targs,)
        else:
            self.orig_loss, targs = self.learn.loss.detach().clone(), *self.learn.yb
        
        cpu_targs = targs.cpu().int().numpy()
        if self.sample_count >= self.total_samples:
            # Enque and deque ID features
            for idx, targ in enumerate(cpu_targs):
                self.features[targ] = torch.cat((self.features[targ][1:], self.batch_features[idx].view(1, -1)), dim=0)

            if self.learn.pct_train >= self.start_pct:
                # the covariance finder needs the data to be centered
                fs = self.features.shape
                mean_embed_id = self.features.mean(1)
                X = (self.features - mean_embed_id[:, None, :]).view(fs[0]*fs[1], fs[2])
                
                # add the variance.
                temp_precision = torch.mm(X.t(), X) / len(X)
                temp_precision += 0.0001 * self.eye_matrix
                
                # gausian sample ood features
                ood_samples = self._sample_dist(mean_embed_id, temp_precision, self.sample_from, 
                                                self.energy_samples, self.nf, self.n_out)

                id_energy = self._log_sum_exp(self.learn.pred, 1)
                ood_pred = self.learn.model[1].head(ood_samples)
                ood_energy = self._log_sum_exp(ood_pred, 1)

                self.energy_pred = torch.cat((id_energy, ood_energy), -1)
                self.energy_targ = torch.cat((torch.ones(len(id_energy), device=self.device, dtype=torch.long), 
                                              torch.zeros(len(ood_pred), device=self.device, dtype=torch.long)), -1)

                self.energy_pred = self.learn.model[1].log_reg(self.energy_pred.view(-1, 1))
                self.energy_loss = self.energy_loss_weight * self.energy_loss_func(self.energy_pred, self.energy_targ)
                self.learn.loss_grad += self.energy_loss
                self.learn.loss += self.energy_loss.detach()

        else:
            for idx, targ in enumerate(cpu_targs):
                if self.number_dict[targ] < self.n_sample:
                    self.features[targ][self.number_dict[targ]] = self.batch_features[idx]
                    self.number_dict[targ] += 1
                    self.sample_count += 1
                else:
                    self.features[targ] = torch.cat((self.features[targ][1:], self.batch_features[idx].view(1, -1)), dim=0)
            self.energy_loss = torch.tensor(0., device=self.device)

    def _vos_valid_step(self):
        self.orig_loss = self.learn.loss.detach().clone()
        if self.eval_vos or (self.learn.pct_train >= self.start_pct and self.sample_count >= self.total_samples):
            id_energy = self._log_sum_exp(self.learn.pred, 1)
            if self.targ_vos: self.energy_targ = self.vos_targs
            else:             self.energy_targ = torch.ones(len(id_energy), device=self.device, dtype=torch.long)
            self.energy_pred = self.learn.model[1].log_reg(id_energy.view(-1, 1))

            self.energy_loss = self.energy_loss_weight * self.energy_loss_func(self.energy_pred, self.energy_targ)
            self.learn.loss += self.energy_loss.detach()
        else:
            self.energy_loss = torch.tensor(0., device=self.device)
            self.energy_pred = torch.tensor(0., device=self.device)
            self.energy_targ = torch.tensor(1., device=self.device)

    # def after_batch(self):
    #     if self.learn.state==LearnerState.Eval or (self.learn.pct_train >= self.start_pct and self.sample_count >= self.total_samples):
    #         self.learn.pred = self.learn.pred, self.energy_pred

    def _log_sum_exp(self, value, dim=None, keepdim=False):
        "Numerically stable implementation of the operation value.exp().sum(dim, keepdim).log()"
        # TODO: torch.max(value, dim=None) threw an error at time of writing
        if dim is not None:
            m, _ = torch.max(value, dim=dim, keepdim=True)
            value = value - m
            if keepdim is False:
                m = m.squeeze(dim)
            return m + torch.log(torch.sum(F.relu(self.learn.model[1].weight_energy.weight) * torch.exp(value), dim=dim, keepdim=keepdim))
        else:
            m = torch.max(value)
            sum_exp = torch.sum(torch.exp(value - m))
            return m + torch.log(sum_exp)

## VOS Metrics -

In [ ]:
#exporti
from fastai.learner import Metric, AvgMetric, ActivationType, MetricType, LogMetric, find_bs, AvgLoss, AvgSmoothLoss, to_detach
from fastai.metrics import accuracy

In [ ]:
#exporti
class VOSMetric(Metric):
    "Blueprint for defining a metric with accumulate"

    def accumulate(self, learn):
        self.pred, self.targ = learn.vos.energy_pred, learn.vos.energy_targ
        if self.activation in [ActivationType.Softmax, ActivationType.BinarySoftmax]:
            self.pred = F.softmax(self.pred, dim=self.dim_argmax)
            if self.activation == ActivationType.BinarySoftmax: self.pred = self.pred[:, -1]
        elif self.activation == ActivationType.Sigmoid: self.pred = torch.sigmoid(self.pred)
        elif self.dim_argmax: self.pred = self.pred.argmax(dim=self.dim_argmax)
        if self.thresh: self.pred = (self.pred >= self.thresh)

In [ ]:
#exporti
@delegates(VOSMetric)
class VOSAvgMetric(VOSMetric):
    "Average the values of `func` taking into account potential different batch sizes"
    def __init__(self, func, to_np=False, invert_arg=False, **kwargs):
        super().__init__(**self._split_kwargs(VOSMetric.__init__, **kwargs))
        self.func, self.fkwargs = func, self._split_kwargs(func, **kwargs)
        self.to_np, self.invert_arg = to_np, invert_arg
        self._name = ifnone(kwargs.get('name', None), self.func.func.__name__ if hasattr(self.func, 'func') else self.func.__name__)

    def reset(self): self.total,self.count = 0.,0

    def accumulate(self, learn):
        super().accumulate(learn)
        bs = find_bs(learn.yb)
        if self.to_np: self.pred,self.targ = learn.to_detach(self.pred).numpy(),learn.to_detach(self.targ).numpy()
        self.total += (self.func(self.targ, self.pred, **self.fkwargs) if self.invert_arg else self.func(self.pred, self.targ, **self.fkwargs))*bs
        self.count += bs
    
    @property
    def value(self): return self.total/self.count if self.count != 0 else None

In [ ]:
#exporti
@delegates(VOSMetric)
def func_to_metric(func, metric_type, is_class, thresh=None, axis=-1, activation=None, log_metric=LogMetric.Valid, **kwargs):
    "Convert `func` metric to a fastai metric"

    dim_argmax = axis if is_class and thresh is None else None
    if activation is None:
        activation = ActivationType.Sigmoid if (is_class and thresh is not None) else ActivationType.No

    if metric_type==MetricType.Avg:
        return VOSAvgMetric(func, dim_argmax=dim_argmax, activation=activation,
                         thresh=thresh, log_metric=log_metric, **kwargs)
    else:
        name = func.func.__name__ if hasattr(func, 'func') else  func.__name__
        raise ValueError(f"Unsupported `metric_type` {metric_type} for metric {name}.")

In [ ]:
#export
def VOSAccuracy(axis=-1, metric_type=MetricType.Avg, log_metric=LogMetric.Valid, **kwargs):
    "Compute accuracy with `targ` when `pred` is bs * n_classes"
    return func_to_metric(accuracy, metric_type, True, axis=axis, log_metric=log_metric, name='vos_accuracy', **kwargs)

In [ ]:
#exporti
class VOSAvgLoss(AvgLoss):
    "Average the MultiLoss losses taking into account potential different batch sizes"
    def __init__(self, name='vos_loss'): 
        self._name = name
    
    def accumulate(self, learn):
        bs = find_bs(learn.yb)
        self.total += learn.to_detach(learn.vos.energy_loss.mean())*bs
        self.count += bs

In [ ]:
#exporti
class OrigAvgLoss(AvgLoss):
    "Average the MultiLoss losses taking into account potential different batch sizes"
    def __init__(self, name='orig_loss'):
        self._name = name
    
    def accumulate(self, learn):
        bs = find_bs(learn.yb)
        self.total += learn.to_detach(learn.vos.orig_loss.mean())*bs
        self.count += bs

In [ ]:
#exporti
class VOSAvgSmoothLoss(AvgSmoothLoss):
    "Smooth average of the MultiLoss losses (exponentially weighted with `beta`)"
    def __init__(self, name='vos_loss', beta=0.98):
        super().__init__(beta)
        self._name = name

    def accumulate(self, learn):
        self.count += 1
        self.val = torch.lerp(to_detach(learn.vos.energy_loss.mean(), gather=False), self.val, self.beta)

In [ ]:
#exporti
class OrigAvgSmoothLoss(AvgSmoothLoss):
    "Smooth average of the MultiLoss losses (exponentially weighted with `beta`)"
    def __init__(self, name='orig_loss', beta=0.98): 
        super().__init__(beta)
        self._name = name

    def accumulate(self, learn):
        self.count += 1
        self.val = torch.lerp(to_detach(learn.vos.orig_loss.mean(), gather=False), self.val, self.beta)

## VOS Learner -

In [ ]:
#export
@delegates(create_vos_model)
def vos_learner(dls, arch, n_out=None, pretrained=True, fastai_head=False, vos_metrics=[VOSAccuracy()],
                # vos args
                start_epoch:int|None=None, start_pct:float=0.40, n_sample:int=1000, sample_from:int=10000,
                energy_samples:int=1, energy_loss_weight:float=0.1, energy_loss_func:nn.Module|None=None,
                dist_sample:VOSDistSample=VOSDistSample.Fork,
                # learner args
                loss_func=None, opt_func=Adam, lr=defaults.lr, splitter=None, cbs=None, metrics=None, path=None,
                model_dir='models', wd=None, wd_bn_bias=False, train_bn=True, moms=(0.95,0.85,0.95),
                # other model args
                **kwargs):
    "Build a convnet style learner from `dls` and `arch`"
    meta = model_meta.get(arch, _default_meta)

    if n_out is None: n_out = get_c(dls)
    assert n_out, "`n_out` is not defined, and could not be inferred from data, set `dls.c` or pass `n_out`"
    model = create_vos_model(arch, n_out, pretrained=pretrained, fastai_head=fastai_head, **kwargs)

    if metrics is None: metrics = L()
    else: metrics=L(metrics)
    metrics = L([OrigAvgSmoothLoss(), VOSAvgSmoothLoss(), OrigAvgLoss(), VOSAvgLoss()]) + metrics
    for m in vos_metrics:
        if m not in metrics:
            metrics.append(m)
    
    vos_cb = [VOSCallback(n_out, start_epoch, start_pct, n_sample, sample_from, 
                          energy_samples, energy_loss_weight, energy_loss_func, dist_sample)]
    if cbs is None: cbs = vos_cb
    else: cbs += vos_cb

    splitter=ifnone(splitter, meta['split'])
    learn = Learner(dls=dls, model=model, loss_func=loss_func, opt_func=opt_func, lr=lr, splitter=splitter, cbs=cbs,
                   metrics=metrics, path=path, model_dir=model_dir, wd=wd, wd_bn_bias=wd_bn_bias, train_bn=train_bn,
                   moms=moms)
    if pretrained: learn.freeze()
    # keep track of args for loggers
    store_attr('arch,pretrained', self=learn, **kwargs)
    return learn